# ETL Transform Notebook

## Contents
1. Import Required Libraries 
2. Load Raw Dataset    
3. Initial Diagnostics  
4. Semantic Zero Detection  
5. Imputation Strategy Space 
6. Reduced Strategy Benchmarking
7. Focused Imputation on 'gap' Feature
8. Metric Summary and Best Config Identification
9. Save Imputed Dataset

## 1. Import Required Libraries
Load essential packages for data access, manipulation, and file handling.

In [2]:
# import statements
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
import itertools

## 2. Load Raw Dataset  
Read the raw earthquake-tsunami dataset into a DataFrame for transformation and imputation benchmarking.

In [3]:
# Load dataset from raw folder
df = pd.read_csv("../data/raw/earthquake_data_tsunami.csv")

## 3. Initial Diagnostics  
Inspect feature types, missing values, value ranges, and potential semantic zeros.

In [4]:
# Check feature types
df.dtypes

magnitude    float64
cdi            int64
mmi            int64
sig            int64
nst            int64
dmin         float64
gap          float64
depth        float64
latitude     float64
longitude    float64
Year           int64
Month          int64
tsunami        int64
dtype: object

In [5]:
# Check for missing values
df.isna().any()

magnitude    False
cdi          False
mmi          False
sig          False
nst          False
dmin         False
gap          False
depth        False
latitude     False
longitude    False
Year         False
Month        False
tsunami      False
dtype: bool

In [6]:
# Check value ranges for each column
[[col, np.min(df[col]), np.max(df[col])] for col in df]

[['magnitude', 6.5, 9.1],
 ['cdi', 0, 9],
 ['mmi', 1, 9],
 ['sig', 650, 2910],
 ['nst', 0, 934],
 ['dmin', 0.0, 17.654],
 ['gap', 0.0, 239.0],
 ['depth', 2.7, 670.81],
 ['latitude', -61.8484, 71.6312],
 ['longitude', -179.968, 179.662],
 ['Year', 2001, 2022],
 ['Month', 1, 12],
 ['tsunami', 0, 1]]

In [7]:
# Check column names
df.columns

Index(['magnitude', 'cdi', 'mmi', 'sig', 'nst', 'dmin', 'gap', 'depth',
       'latitude', 'longitude', 'Year', 'Month', 'tsunami'],
      dtype='object')

Check 0 value in coordinates. None found.

In [8]:
# Check for 0 values in coordinates
print((df["latitude"] == 0).any())
print((df["longitude"] == 0).any())

False
False


### 3.1 Descriptive statistics

CDI, NST: More than 25% of the values are 0. Dmin: More than 50% of the values are 0. We could consider binning and or imputation.

In [9]:
# Summary statistics
df.describe(include="all")

,magnitude,cdi,mmi,sig,nst,dmin,gap,depth,latitude,longitude,Year,Month,tsunami
count,782.000000,782.000000,782.000000,782.000000,782.000000,782.000000,782.000000,782.000000,782.000000,782.000000,782.000000,782.000000,782.000000
mean,6.941125,4.333760,5.964194,870.108696,230.250639,1.325757,25.038990,75.883199,3.538100,52.609199,2012.280051,6.563939,0.388747
std,0.445514,3.169939,1.462724,322.465367,250.188177,2.218805,24.225067,137.277078,27.303429,117.898886,6.099439,3.507866,0.487778
min,6.500000,0.000000,1.000000,650.000000,0.000000,0.000000,0.000000,2.700000,-61.848400,-179.968000,2001.000000,1.000000,0.000000
25%,6.600000,0.000000,5.000000,691.000000,0.000000,0.000000,14.625000,14.000000,-14.595600,-71.668050,2007.000000,3.250000,0.000000
50%,6.800000,5.000000,6.000000,754.000000,140.000000,0.000000,20.000000,26.295000,-2.572500,109.426000,2013.000000,7.000000,0.000000
75%,7.100000,7.000000,7.000000,909.750000,445.000000,1.863000,30.000000,49.750000,24.654500,148.941000,2017.000000,10.000000,1.000000
max,9.100000,9.000000,9.000000,2910.000000,934.000000,17.654000,239.000000,670.810000,71.631200,179.662000,2022.000000,12.000000,1.000000


Skew and kurtosis for distribution shape

In [10]:
df.skew()

magnitude    1.444440
cdi         -0.197310
mmi         -0.250403
sig          3.083629
nst          0.533307
dmin         2.604580
gap          4.668607
depth        3.024869
latitude     0.200853
longitude   -0.702982
Year        -0.192450
Month       -0.067928
tsunami      0.457333
dtype: float64

In [11]:
df.kurtosis()

magnitude     2.226391
cdi          -1.357753
mmi          -0.224592
sig          12.000754
nst          -1.092793
dmin          9.283367
gap          32.027722
depth         8.384480
latitude     -0.476740
longitude    -1.088383
Year         -1.042840
Month        -1.299853
tsunami      -1.795445
dtype: float64

## 4. Semantic Zero Detection
Replace 0s in selected features with `NaN` to prepare for imputation.  
Features: `cdi`, `nst`, `dmin`, `gap`

In [12]:
# Replace semantic zeros with NaN
df_nan = df.copy()
for col in ["cdi", "nst", "dmin", "gap"]:
    df_nan[col] = df_nan[col].replace(0, np.nan)

## 5. Imputation Strategy Space  
Define strategy options for each feature: none, mean, KNN, KMeans.  
Generate all permutations for benchmarking.


I will consider number of clusters and neighbours in the range 2 to 10

In [13]:
feature_discrete_counts = {
    col: df_nan[col].dropna().nunique()
    for col in ["cdi", "nst", "dmin", "gap"]
}

In [14]:
feature_discrete_counts

{'cdi': 9, 'nst': 311, 'dmin': 368, 'gap': 255}

We set our stategues and range for imputation

In [15]:
# Strategy space and feature list
import itertools
strategy_space = (
    [("none", None)] +
    [("mean", None)] +
    [("knn", i) for i in range(2, 11)] +
    [("kmeans", i) for i in range(2, 11)]
)
features = ["cdi", "nst", "dmin", "gap"]

# Full config space
all_configs = list(itertools.product(strategy_space, repeat=len(features)))
len(all_configs)

160000

It will take a long time to fit 160k models simply for deciding on imputation

## 6. Reduced Strategy Benchmarking  
Limit strategy space for tractability.  
Train logistic regression on each config and log metrics.

In [16]:
# Reduced strategy space
reduced_strategy_space = (
    [("none", None)] +
    [("mean", None)] +
    [("knn", i) for i in range(2, 4)] +
    [("kmeans", i) for i in range(2, 4)]
)
# Create all combinations: one strategy-param pair per feature
reduced_configs = list(itertools.product(reduced_strategy_space, repeat=len(features)))
len(reduced_configs)

1296

A reasonable number of permutations to model.

In [17]:
def apply_imputation(df: pd.DataFrame, plan: dict) -> pd.DataFrame:
    df_copy = df.copy()

    for feature, (strategy, param) in plan.items():
        if strategy == "none":
            df_copy[feature].fillna(-999, inplace=True)  # fill na with sentinel value

        elif strategy == "mean":
            imputer = SimpleImputer(strategy="mean")
            df_copy[[feature]] = imputer.fit_transform(df_copy[[feature]])

        elif strategy == "knn":
            # KNNImputer expects multiple columns, so we isolate and reinsert
            imputer = KNNImputer(n_neighbors=param)
            df_copy[[feature]] = imputer.fit_transform(df_copy[[feature]])

        elif strategy == "kmeans":
            missing_mask = df_copy[feature].isna()
            observed = df_copy.loc[~missing_mask, feature].values.reshape(-1, 1)

            if len(observed) < param:
                # Not enough data to form clusters — fallback to mean
                fill_value = np.nanmean(observed)
            else:
                kmeans = KMeans(n_clusters=param, random_state=42)
                kmeans.fit(observed)
                centers = kmeans.cluster_centers_
                # Assign missing values to nearest center (or random center)
                fill_value = np.random.choice(centers.flatten())

            df_copy.loc[missing_mask, feature] = fill_value

    return df_copy

In [18]:
from sklearn.model_selection import train_test_split

def split_data(df: pd.DataFrame, target_col: str = "target", test_size: float = 0.2, random_state: int = 42):
    X = df.drop(columns=[target_col])
    y = df[target_col]
    
    return train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, brier_score_loss
from sklearn.impute import KNNImputer

results = []

for config in reduced_configs:
    impute_plan = dict(zip(features, config))
    df_imputed = apply_imputation(df_nan, impute_plan)  # Your custom imputer
    
    X_train, X_valid, y_train, y_valid = split_data(df_imputed, target_col="tsunami")
    model = LogisticRegression().fit(X_train, y_train)
    y_prob = model.predict_proba(X_valid)[:, 1]
    
    # Fixed threshold for now (can tune later)
    y_pred = (y_prob >= 0.5).astype(int)
    
    metrics = {
        "config": impute_plan,
        "auc": roc_auc_score(y_valid, y_prob),
        "f1": f1_score(y_valid, y_pred),
        "precision": precision_score(y_valid, y_pred),
        "recall": recall_score(y_valid, y_pred),
        "brier": brier_score_loss(y_valid, y_prob)
    }
    
    results.append(metrics)

c:\Users\Daniel\vsc-projects\codeinstitute\projects\tsunami-earthquake-analysis\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Daniel\vsc-projects\codeinstitute\projects\tsunami-earthquake-analysis\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation f

In [20]:
len(results)

1296

In [21]:
metrics = ["auc", "f1", "precision", "recall", "brier"]

for metric in metrics:
    values = [r[metric] for r in results]
    print(f"{metric.upper():<9} → min: {min(values):.4f}, max: {max(values):.4f}, range: {max(values) - min(values):.4f}")

AUC       → min: 0.5488, max: 0.8841, range: 0.3352
F1        → min: 0.2667, max: 0.8271, range: 0.5604
PRECISION → min: 0.3721, max: 0.7647, range: 0.3926
RECALL    → min: 0.1967, max: 0.9016, range: 0.7049
BRIER     → min: 0.1202, max: 0.2452, range: 0.1250


In [22]:
results[0]

{'config': {'cdi': ('none', None),
  'nst': ('none', None),
  'dmin': ('none', None),
  'gap': ('none', None)},
 'auc': 0.8755122950819672,
 'f1': 0.7938931297709924,
 'precision': 0.7428571428571429,
 'recall': 0.8524590163934426,
 'brier': 0.12319744054857214}

In [23]:
best_recall = max(results, key=lambda x: x['recall'])

In [24]:
best_recall

{'config': {'cdi': ('none', None),
  'nst': ('mean', None),
  'dmin': ('none', None),
  'gap': ('kmeans', 3)},
 'auc': 0.8705601092896176,
 'f1': 0.8208955223880597,
 'precision': 0.7534246575342466,
 'recall': 0.9016393442622951,
 'brier': 0.12245643369161478}

## 7. Focused Imputation on 'gap' Feature  
Fix strategies for `cdi`, `nst`, `dmin` and vary `gap` to isolate impact.

In [25]:
# Fixed strategies for first three features
fixed = [("none", None), ("mean", None), ("none", None)]
gap_config = []

for method in ["knn", "kmeans"]:
    for k in range(2, 11):
        gap = (method, k)
        config = tuple(fixed + [gap])
        gap_config.append(config)

len(gap_config)

18

In [26]:
gap_results = []

for config in gap_config:
    impute_plan = dict(zip(features, config))
    df_imputed = apply_imputation(df_nan, impute_plan)  # Your custom imputer
    
    X_train, X_valid, y_train, y_valid = split_data(df_imputed, target_col="tsunami")
    model = LogisticRegression().fit(X_train, y_train)
    y_prob = model.predict_proba(X_valid)[:, 1]
    
    # Fixed threshold for now (can tune later)
    y_pred = (y_prob >= 0.5).astype(int)
    
    metrics = {
        "config": impute_plan,
        "auc": roc_auc_score(y_valid, y_prob),
        "f1": f1_score(y_valid, y_pred),
        "precision": precision_score(y_valid, y_pred),
        "recall": recall_score(y_valid, y_pred),
        "brier": brier_score_loss(y_valid, y_prob)
    }
    
    gap_results.append(metrics)

c:\Users\Daniel\vsc-projects\codeinstitute\projects\tsunami-earthquake-analysis\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Daniel\vsc-projects\codeinstitute\projects\tsunami-earthquake-analysis\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation f

The metrics are similar to before.

## 8. Metric Summary and Best Config Identification  
Evaluate AUC, F1, precision, recall, and Brier score.  
Identify best-performing config by recall.

In [27]:
# Summarize metrics across configs
metrics = ["auc", "f1", "precision", "recall", "brier"]
for metric in metrics:
    values = [r[metric] for r in gap_results]
    print(f"{metric.upper():<9} → min: {min(values):.4f}, max: {max(values):.4f}, range: {max(values) - min(values):.4f}")

# Best config by recall
best_recall_gap = max(gap_results, key=lambda x: x['recall'])
best_recall_gap

AUC       → min: 0.8700, max: 0.8728, range: 0.0027
F1        → min: 0.8120, max: 0.8209, range: 0.0089
PRECISION → min: 0.7500, max: 0.7534, range: 0.0034
RECALL    → min: 0.8852, max: 0.9016, range: 0.0164
BRIER     → min: 0.1220, max: 0.1226, range: 0.0005


{'config': {'cdi': ('none', None),
  'nst': ('mean', None),
  'dmin': ('none', None),
  'gap': ('kmeans', 3)},
 'auc': 0.8727800546448087,
 'f1': 0.8208955223880597,
 'precision': 0.7534246575342466,
 'recall': 0.9016393442622951,
 'brier': 0.122040507233138}

The model with the best recall was again (none, mean, none, kmeans3)

## 9. Save Imputed Dataset
Apply best imputation config and persist to `/data/processed/` for downstream feature engineering.

In [29]:
# Apply best config
best_plan = {'cdi': ('none', None), 'nst': ('mean', None), 'dmin': ('none', None), 'gap': ('kmeans', 3)}
df_imputed = apply_imputation(df_nan, best_plan)

# Save to processed folder
df_imputed.to_csv("../data/processed/earthquake_imputed.csv", index=False)

c:\Users\Daniel\vsc-projects\codeinstitute\projects\tsunami-earthquake-analysis\.venv\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
